I trained roberta-large model, got local f1 Score: 0.8680, but I use this model to submit got 0.807, what's your roberta local vs lb gap? is it that big? 


Thanks

- https://www.kaggle.com/code/yasufuminakama/nbme-deberta-base-baseline-train
- https://www.kaggle.com/code/theoviel/roberta-strikes-back

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import sys
import random
import ast
import numpy as np
import torch
import transformers
import torch.nn as nn
import transformers
from tqdm import tqdm
from sklearn.metrics import f1_score
import itertools
import math
from datetime import datetime
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoConfig, AutoModel
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

%env TOKENIZERS_PARALLELISM=true

In [ ]:
class CFG:
    wandb = False
    competition = 'NBME'
    _wandb_kernel = 'nakama'
    debug = False
    apex = True
    print_freq = 100
    num_workers = 2
    model = "roberta-large"
    scheduler = 'cosine'  # ['linear', 'cosine']
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    epochs = 5
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    eps = 1e-6
    betas = (0.9, 0.999)
    batch_size = 4
    fc_dropout = 0.2
    max_len = 466
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    precompute_tokens = True
    seed = 42
    n_fold = 5
    trn_fold = [0]
    data_config = {
        "val_bs": 16 if "large" in model else 32,
        "pad_token": 1 if "roberta" in model else 0,
    }
    device = "cuda"
    DATA_PATH = "../input/nbme-score-clinical-patient-notes/"
    OUTPUT_DIR = './'
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        
        
model_path = {
    'roberta-base': '../input/roberta-base',
    'roberta-large': '../input/robertalarge'
}

In [ ]:

def get_logger(filename):
    import logging
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        handlers=[logging.StreamHandler(sys.stdout)],
    )
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    #     handler1 = StreamHandler()
    #     handler1.setFormatter(Formatter("%(message)s"))
    exp = datetime.now().strftime('%Y%m%d-%H%M%S')
    handler2 = FileHandler(filename=f"{filename}_{exp}.log")
    print('log file', f"{filename}_{exp}.log")
    #     handler2.setFormatter(Formatter("%(message)s"))
    #     logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features


def clean_data(train):
    # incorrect annotation
    train.loc[338,
              'annotation'] = ast.literal_eval('[["father heart attack"]]')
    train.loc[338, 'location'] = ast.literal_eval('[["764 783"]]')

    train.loc[621,
              'annotation'] = ast.literal_eval('[["for the last 2-3 months"]]')
    train.loc[621, 'location'] = ast.literal_eval('[["77 100"]]')

    train.loc[655, 'annotation'] = ast.literal_eval(
        '[["no heat intolerance"], ["no cold intolerance"]]')
    train.loc[655, 'location'] = ast.literal_eval(
        '[["285 292;301 312"], ["285 287;296 312"]]')

    train.loc[1262,
              'annotation'] = ast.literal_eval('[["mother thyroid problem"]]')
    train.loc[1262, 'location'] = ast.literal_eval('[["551 557;565 580"]]')

    train.loc[1265, 'annotation'] = ast.literal_eval(
        '[[\'felt like he was going to "pass out"\']]')
    train.loc[1265, 'location'] = ast.literal_eval('[["131 135;181 212"]]')

    train.loc[1396,
              'annotation'] = ast.literal_eval('[["stool , with no blood"]]')
    train.loc[1396, 'location'] = ast.literal_eval('[["259 280"]]')

    train.loc[1591,
              'annotation'] = ast.literal_eval('[["diarrhoe non blooody"]]')
    train.loc[1591, 'location'] = ast.literal_eval('[["176 184;201 212"]]')

    train.loc[1615, 'annotation'] = ast.literal_eval(
        '[["diarrhea for last 2-3 days"]]')
    train.loc[1615, 'location'] = ast.literal_eval('[["249 257;271 288"]]')

    train.loc[1664,
              'annotation'] = ast.literal_eval('[["no vaginal discharge"]]')
    train.loc[1664, 'location'] = ast.literal_eval('[["822 824;907 924"]]')

    train.loc[1714, 'annotation'] = ast.literal_eval(
        '[["started about 8-10 hours ago"]]')
    train.loc[1714, 'location'] = ast.literal_eval('[["101 129"]]')

    train.loc[1929,
              'annotation'] = ast.literal_eval('[["no blood in the stool"]]')
    train.loc[1929, 'location'] = ast.literal_eval('[["531 539;549 561"]]')

    train.loc[2134, 'annotation'] = ast.literal_eval(
        '[["last sexually active 9 months ago"]]')
    train.loc[2134, 'location'] = ast.literal_eval('[["540 560;581 593"]]')

    train.loc[2191, 'annotation'] = ast.literal_eval(
        '[["right lower quadrant pain"]]')
    train.loc[2191, 'location'] = ast.literal_eval('[["32 57"]]')

    train.loc[2553,
              'annotation'] = ast.literal_eval('[["diarrhoea no blood"]]')
    train.loc[2553, 'location'] = ast.literal_eval('[["308 317;376 384"]]')

    train.loc[3124, 'annotation'] = ast.literal_eval('[["sweating"]]')
    train.loc[3124, 'location'] = ast.literal_eval('[["549 557"]]')

    train.loc[3858, 'annotation'] = ast.literal_eval(
        '[["previously as regular"], ["previously eveyr 28-29 days"], ["previously lasting 5 days"], ["previously regular flow"]]'
    )
    train.loc[3858, 'location'] = ast.literal_eval(
        '[["102 123"], ["102 112;125 141"], ["102 112;143 157"], ["102 112;159 171"]]'
    )

    train.loc[4373, 'annotation'] = ast.literal_eval('[["for 2 months"]]')
    train.loc[4373, 'location'] = ast.literal_eval('[["33 45"]]')

    train.loc[4763, 'annotation'] = ast.literal_eval('[["35 year old"]]')
    train.loc[4763, 'location'] = ast.literal_eval('[["5 16"]]')

    train.loc[4782,
              'annotation'] = ast.literal_eval('[["darker brown stools"]]')
    train.loc[4782, 'location'] = ast.literal_eval('[["175 194"]]')

    train.loc[4908,
              'annotation'] = ast.literal_eval('[["uncle with peptic ulcer"]]')
    train.loc[4908, 'location'] = ast.literal_eval('[["700 723"]]')

    train.loc[6016, 'annotation'] = ast.literal_eval(
        '[["difficulty falling asleep"]]')
    train.loc[6016, 'location'] = ast.literal_eval('[["225 250"]]')

    train.loc[6192, 'annotation'] = ast.literal_eval(
        '[["helps to take care of aging mother and in-laws"]]')
    train.loc[6192, 'location'] = ast.literal_eval('[["197 218;236 260"]]')

    train.loc[6380, 'annotation'] = ast.literal_eval(
        '[["No hair changes"], ["No skin changes"], ["No GI changes"], ["No palpitations"], ["No excessive sweating"]]'
    )
    train.loc[6380, 'location'] = ast.literal_eval(
        '[["480 482;507 519"], ["480 482;499 503;512 519"], ["480 482;521 531"], ["480 482;533 545"], ["480 482;564 582"]]'
    )

    train.loc[6562, 'annotation'] = ast.literal_eval(
        '[["stressed due to taking care of her mother"], ["stressed due to taking care of husbands parents"]]'
    )
    train.loc[6562, 'location'] = ast.literal_eval(
        '[["290 320;327 337"], ["290 320;342 358"]]')

    train.loc[6862, 'annotation'] = ast.literal_eval(
        '[["stressor taking care of many sick family members"]]')
    train.loc[6862, 'location'] = ast.literal_eval('[["288 296;324 363"]]')

    train.loc[7022, 'annotation'] = ast.literal_eval(
        '[["heart started racing and felt numbness for the 1st time in her finger tips"]]'
    )
    train.loc[7022, 'location'] = ast.literal_eval('[["108 182"]]')

    train.loc[7422,
              'annotation'] = ast.literal_eval('[["first started 5 yrs"]]')
    train.loc[7422, 'location'] = ast.literal_eval('[["102 121"]]')

    train.loc[8876,
              'annotation'] = ast.literal_eval('[["No shortness of breath"]]')
    train.loc[8876, 'location'] = ast.literal_eval('[["481 483;533 552"]]')

    train.loc[9027, 'annotation'] = ast.literal_eval(
        '[["recent URI"], ["nasal stuffines, rhinorrhea, for 3-4 days"]]')
    train.loc[9027, 'location'] = ast.literal_eval('[["92 102"], ["123 164"]]')

    train.loc[9938, 'annotation'] = ast.literal_eval(
        '[["irregularity with her cycles"], ["heavier bleeding"], ["changes her pad every couple hours"]]'
    )
    train.loc[9938, 'location'] = ast.literal_eval(
        '[["89 117"], ["122 138"], ["368 402"]]')

    train.loc[9973, 'annotation'] = ast.literal_eval('[["gaining 10-15 lbs"]]')
    train.loc[9973, 'location'] = ast.literal_eval('[["344 361"]]')

    train.loc[10513, 'annotation'] = ast.literal_eval(
        '[["weight gain"], ["gain of 10-16lbs"]]')
    train.loc[10513,
              'location'] = ast.literal_eval('[["600 611"], ["607 623"]]')

    train.loc[11551, 'annotation'] = ast.literal_eval(
        '[["seeing her son knows are not real"]]')
    train.loc[11551, 'location'] = ast.literal_eval('[["386 400;443 461"]]')

    train.loc[11677, 'annotation'] = ast.literal_eval(
        '[["saw him once in the kitchen after he died"]]')
    train.loc[11677, 'location'] = ast.literal_eval('[["160 201"]]')

    train.loc[12124, 'annotation'] = ast.literal_eval(
        '[["tried Ambien but it didnt work"]]')
    train.loc[12124, 'location'] = ast.literal_eval('[["325 337;349 366"]]')

    train.loc[12279, 'annotation'] = ast.literal_eval(
        '[["heard what she described as a party later than evening these things did not actually happen"]]'
    )
    train.loc[12279, 'location'] = ast.literal_eval('[["405 459;488 524"]]')

    train.loc[12289, 'annotation'] = ast.literal_eval(
        '[["experienced seeing her son at the kitchen table these things did not actually happen"]]'
    )
    train.loc[12289, 'location'] = ast.literal_eval('[["353 400;488 524"]]')

    train.loc[13238, 'annotation'] = ast.literal_eval(
        '[["SCRACHY THROAT"], ["RUNNY NOSE"]]')
    train.loc[13238,
              'location'] = ast.literal_eval('[["293 307"], ["321 331"]]')

    train.loc[13297, 'annotation'] = ast.literal_eval(
        '[["without improvement when taking tylenol"], ["without improvement when taking ibuprofen"]]'
    )
    train.loc[13297, 'location'] = ast.literal_eval(
        '[["182 221"], ["182 213;225 234"]]')

    train.loc[13299, 'annotation'] = ast.literal_eval(
        '[["yesterday"], ["yesterday"]]')
    train.loc[13299, 'location'] = ast.literal_eval('[["79 88"], ["409 418"]]')

    train.loc[13845, 'annotation'] = ast.literal_eval(
        '[["headache global"], ["headache throughout her head"]]')
    train.loc[13845, 'location'] = ast.literal_eval(
        '[["86 94;230 236"], ["86 94;237 256"]]')

    train.loc[14083, 'annotation'] = ast.literal_eval(
        '[["headache generalized in her head"]]')
    train.loc[14083, 'location'] = ast.literal_eval('[["56 64;156 179"]]')
    train['annotation_length'] = train['annotation'].apply(len)
    return train


def clean_feature(df):
    df['pn_history'] = df['pn_history'].apply(lambda x: x.strip())
    df['feature_text'] = df['feature_text'].apply(process_feature_text)

    df['feature_text'] = df['feature_text'].apply(clean_spaces)
    df['clean_text'] = df['pn_history'].apply(clean_spaces)
    return df

def process_feature_text(text):
    text = re.sub('I-year', '1-year', text)
    text = re.sub('-OR-', " or ", text)
    text = re.sub('-', ' ', text)
    return text

def clean_spaces(txt):
    txt = re.sub('\n', ' ', txt)
    txt = re.sub('\t', ' ', txt)
    txt = re.sub('\r', ' ', txt)
    #     txt = re.sub(r'\s+', ' ', txt)
    return txt

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def token_pred_to_char_pred(token_pred, offsets):
    char_pred = np.zeros((np.max(offsets), 1))
    for i in range(len(token_pred)):
        s, e = int(offsets[i][0]), int(offsets[i][1])  # start, end
        char_pred[s:e] = token_pred[i]
    return char_pred

def post_process_spaces(target, text):
    target = np.copy(target)

    if len(text) > len(target):
        padding = np.zeros(len(text) - len(target))
        target = np.concatenate([target, padding])
    else:
        target = target[:len(text)]

    if text[0] == " ":
        target[0] = 0
    if text[-1] == " ":
        target[-1] = 0

    for i in range(1, len(text) - 1):
        if text[i] == " ":
            if target[i] and not target[i - 1]:  # space before
                target[i] = 0

            if target[i] and not target[i + 1]:  # space after
                target[i] = 0

            if target[i - 1] and target[i + 1]:
                target[i] = 1

    return target


def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(
            np.max(pred) if len(pred) else 0,
            np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score

def char_target_to_span(char_target):
    spans = []
    start, end = 0, 0
    for i in range(len(char_target)):
        if char_target[i] == 1 and char_target[i - 1] == 0:
            if end:
                spans.append([start, end])
            start = i
            end = i + 1
        elif char_target[i] == 1:
            end = i + 1
        else:
            if end:
                spans.append([start, end])
            start, end = 0, 0
    return spans

## dataset

In [ ]:

def get_tokenizer(name, precompute=False, df=None, folder=None):
    if folder is None:
        tokenizer = AutoTokenizer.from_pretrained(model_path[name])
    else:
        tokenizer = AutoTokenizer.from_pretrained(folder)

    tokenizer.name = name
    tokenizer.special_tokens = {
        "sep": tokenizer.sep_token_id,
        "cls": tokenizer.cls_token_id,
        "pad": tokenizer.pad_token_id,
    }

    if precompute:
        tokenizer.precomputed = precompute_tokens(df, tokenizer)
    else:
        tokenizer.precomputed = None

    return tokenizer


def precompute_tokens(df, tokenizer):
    feature_texts = df["feature_text"].unique()

    ids = {}
    offsets = {}

    for feature_text in feature_texts:
        encoding = tokenizer(
            feature_text,
            return_token_type_ids=True,
            return_offsets_mapping=True,
            return_attention_mask=False,
            add_special_tokens=False,
        )
        ids[feature_text] = encoding["input_ids"]
        offsets[feature_text] = encoding["offset_mapping"]

    texts = df["clean_text"].unique()

    for text in texts:
        encoding = tokenizer(
            text,
            return_token_type_ids=True,
            return_offsets_mapping=True,
            return_attention_mask=False,
            add_special_tokens=False,
        )
        ids[text] = encoding["input_ids"]
        offsets[text] = encoding["offset_mapping"]

    return {"ids": ids, "offsets": offsets}


def encodings_from_precomputed(feature_text, text, char_target, precomputed, tokenizer, max_len=300):
    tokens = tokenizer.special_tokens

    # Input ids
    if "roberta" in tokenizer.name:
        qa_sep = [tokens["sep"], tokens["sep"]]
    else:
        qa_sep = [tokens["sep"]]

    input_ids = [tokens["cls"]] + precomputed["ids"][feature_text] + qa_sep
    n_question_tokens = len(input_ids)

    input_ids += precomputed["ids"][text]
    input_ids = input_ids[: max_len - 1] + [tokens["sep"]]

    # Token type ids
    if "roberta" not in tokenizer.name:
        token_type_ids = np.ones(len(input_ids))
        token_type_ids[:n_question_tokens] = 0
        token_type_ids = token_type_ids.tolist()
    else:
        token_type_ids = [0] * len(input_ids)

    # Offsets
    offsets = [(0, 0)] * n_question_tokens + precomputed["offsets"][text]
    offsets = offsets[: max_len - 1] + [(0, 0)]

    label = np.zeros(len(offsets))
    label[:n_question_tokens] = -1
    label[-1] = -1
#         location_list = ['696 724']
# [s.split() for s in location.split(';')]
#     for location in char_target:
    for loc in char_target:
        start_idx = -1
        end_idx = -1
        start, end = int(loc[0]), int(loc[1])
        for idx in range(len(offsets)):
            if (start_idx == -1) & (start < offsets[idx][0]):
                start_idx = idx - 1
            if (end_idx == -1) & (end <= offsets[idx][1]):
                end_idx = idx + 1
        if start_idx == -1:
            start_idx = end_idx
        if (start_idx != -1) & (end_idx != -1):
            label[start_idx:end_idx] = 1
    # Padding
    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([tokens["pad"]] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        offsets = offsets + ([(0, 0)] * padding_length)
        label = np.pad(label, (0, padding_length), constant_values=-1)

    encoding = {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "offset_mapping": offsets,
        "label": label,
    }

    return encoding

class PatientNoteDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer

        self.texts = df['clean_text'].values
        self.feature_text = df['feature_text'].values
        self.char_targets = df['target'].values.tolist()

    def __getitem__(self, idx):
        text = self.texts[idx]
        feature_text = self.feature_text[idx]
        char_target = self.char_targets[idx]

        # Tokenize
        if self.tokenizer.precomputed is None:
            encoding = self.tokenizer(
                feature_text,
                text,
                return_token_type_ids=True,
                return_offsets_mapping=True,
                return_attention_mask=False,
                truncation="only_second",
                max_length=self.max_len,
                padding='max_length',
            )
            raise NotImplementedError("fix issues with question offsets")
        else:
            encoding = encodings_from_precomputed(
                feature_text,
                text,
                char_target,
                self.tokenizer.precomputed,
                self.tokenizer,
                max_len=self.max_len
            )
        return {
            "ids": torch.tensor(encoding["input_ids"], dtype=torch.long),
            "token_type_ids": torch.tensor(encoding["token_type_ids"], dtype=torch.long),
            "target": torch.tensor(encoding['label'], dtype=torch.float),
            "offsets": np.array(encoding["offset_mapping"]),
            "text": text,
        }

    def __len__(self):
        return len(self.texts)


## train

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD, AdamW
import gc
import time
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup


In [ ]:

def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler,
             device, CFG):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    #     dict_keys(['ids', 'token_type_ids', 'target', 'offsets', 'text'])

    for step, data in enumerate(train_loader):
        labels = data['target'].to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            outputs = model(data['ids'].to(device),
                            data['token_type_ids'].to(device))
            # loss = outputs['loss']
            y_preds = outputs['logits']
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(),
                                                   CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader) - 1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '.format(
                      epoch + 1,
                      step,
                      len(train_loader),
                      remain=timeSince(start,
                                       float(step + 1) / len(train_loader)),
                      loss=losses,
                      grad_norm=grad_norm,
                      lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({
                f"[fold{fold}] loss": losses.val,
                f"[fold{fold}] lr": scheduler.get_lr()[0]
            })
    return losses.avg


def valid_fn(valid_loader, model, criterion, device, CFG):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, data in enumerate(valid_loader):
        labels = data['target'].to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            outputs = model(data['ids'].to(device),
                            data['token_type_ids'].to(device))
            # loss = outputs['loss']
            y_preds = outputs['logits']
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader) - 1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '.format(
                      step,
                      len(valid_loader),
                      loss=losses,
                      remain=timeSince(start,
                                       float(step + 1) / len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions


def train_loop(folds, fold, tokenizer, CFG, LOGGER):
    LOGGER.info(f"========== fold: {fold} training ==========")
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    if CFG.debug:
        LOGGER.info('debug mode')
        train_folds = train_folds.head(100)
        valid_folds = valid_folds.head(100)
#     valid_texts = valid_folds['pn_history'].values
#     valid_labels = create_labels_for_scoring(valid_folds)

    #     train_dataset = TrainDataset(CFG, train_folds)
    #     valid_dataset = TrainDataset(CFG, valid_folds)
    train_dataset = PatientNoteDataset(
        train_folds,
        tokenizer,
        max_len=CFG.max_len,
    )
    valid_dataset = PatientNoteDataset(
        valid_folds,
        tokenizer,
        max_len=CFG.max_len,
    )
    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers,
                              pin_memory=True,
                              drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers,
                              pin_memory=True,
                              drop_last=False)
    offsets = [x['offsets'] for x in valid_dataset]
    offsets = np.array(offsets)
    # ====================================================
    # model & optimizer
    # ====================================================
    model = NERTransformer(CFG.model)
    #     torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(CFG.device)

    # # logits  transformer
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [{
            'params': [
                p for n, p in model.transformer.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            'lr':
            encoder_lr,
            'weight_decay':
            weight_decay
        }, {
            'params': [
                p for n, p in model.transformer.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            'lr':
            encoder_lr,
            'weight_decay':
            0.0
        }, {
            'params':
            [p for n, p in model.named_parameters() if "transformer" not in n],
            'lr':
            decoder_lr,
            'weight_decay':
            0.0
        }]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr,
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters,
                      lr=CFG.encoder_lr,
                      eps=CFG.eps,
                      betas=CFG.betas)

    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=cfg.num_warmup_steps,
                num_training_steps=num_train_steps)
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer,
                num_warmup_steps=cfg.num_warmup_steps,
                num_training_steps=num_train_steps,
                num_cycles=cfg.num_cycles)
        return scheduler

    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.BCEWithLogitsLoss(reduction="none")

    best_score = -1
    df_test = valid_folds.copy()
    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer,
                            epoch, scheduler, CFG.device, CFG)
#         torch.save(model.state_dict(), CFG.OUTPUT_DIR + f'model_{epoch}.pth')
#         LOGGER.info(f'save model model_{epoch}.pth')
        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion,
                                             CFG.device, CFG)
        predictions = predictions.reshape((len(valid_folds), CFG.max_len))

        preds = [
            token_pred_to_char_pred(y, ofs)
            for y, ofs in zip(predictions, offsets)
        ]

        df_test['preds'] = preds
        df_test['preds'] = df_test.apply(
            lambda x: x['preds'][:len(x['clean_text'])], 1)
        df_test['preds'] = df_test['preds'].apply(lambda x:
                                                  (x > 0.5).flatten())
        # df_test['span'] = df_test['preds'].apply(char_target_to_span)
        df_test['preds_pp'] = df_test.apply(
            lambda x: post_process_spaces(x['preds'], x['clean_text']), 1)
        df_test['span'] = df_test['preds_pp'].apply(char_target_to_span)
        score = get_score(df_test['target'], df_test['span'])
        
        elapsed = time.time() - start_time

        LOGGER.info(
            f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s'
        )
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')

        if best_score < score:
            best_score = score
            LOGGER.info(
                f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save(
                {
                    'model': model.state_dict(),
                    'predictions': predictions
                }, CFG.OUTPUT_DIR +
                f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(
        CFG.OUTPUT_DIR + f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
        map_location=torch.device('cpu'))['predictions']
    valid_folds[[i for i in range(CFG.max_len)]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    return valid_folds


## model

In [ ]:
class NERTransformer(nn.Module):
    def __init__(
        self,
        model,
        num_classes=1,
        config_file=None,
        pretrained=True,
    ):
        super().__init__()
        self.name = model
        self.pad_idx = 1 if "roberta" in self.name else 0

        transformers.logging.set_verbosity_error()

        if config_file is None:
            config = AutoConfig.from_pretrained(model_path[model], output_hidden_states=True)
        else:
            config = torch.load(config_file)

        if pretrained:
            self.transformer = AutoModel.from_pretrained(model_path[model], config=config)
        else:
            self.transformer = AutoModel.from_config(config)

        self.nb_features = config.hidden_size

#         self.cnn = nn.Identity()
        self.dropout = nn.Dropout(0.1)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.output = nn.Linear(self.nb_features, num_classes)

    def loss(self, outputs, targets, attention_mask):
        criterion = nn.BCEWithLogitsLoss(reduction='none')
        loss = criterion(outputs.view(-1, 1), targets.view(-1, 1))
        loss = torch.masked_select(loss, targets.view(-1, 1) != -1).mean()

#         active_loss = attention_mask.view(-1) == 1
#         active_logits = outputs.view(-1, self.num_labels)
#         true_labels = targets.view(-1)
#         outputs = active_logits.argmax(dim=-1)
#         idxs = np.where(active_loss.cpu().numpy() == 1)[0]
#         active_logits = active_logits[idxs]
#         true_labels = true_labels[idxs].to(torch.long)
#         loss = loss_fct(active_logits, true_labels)
        return loss

    def forward(self, tokens, token_type_ids, targets=None):
        """
        Usual torch forward function

        Arguments:
            tokens {torch tensor} -- Sentence tokens
            token_type_ids {torch tensor} -- Sentence tokens ids
        """
        mask = (tokens != self.pad_idx).long()
        sequence_output = self.transformer(
            tokens,
            attention_mask=mask,
            token_type_ids=token_type_ids,
        ).last_hidden_state

#         features = hidden_states[-1]
#         logits = self.logits(self.dropout(features))
        logits = self.output(self.dropout(sequence_output))
        # logits1 = self.output(self.dropout1(sequence_output))
        # logits2 = self.output(self.dropout2(sequence_output))
        # logits3 = self.output(self.dropout3(sequence_output))
        # logits4 = self.output(self.dropout4(sequence_output))
        # logits5 = self.output(self.dropout5(sequence_output))

        # logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
#         logits = torch.sigmoid(logits)
        loss = 0
        output = {"logits": logits}
        if targets is not None:
            loss = self.loss(logits, targets, mask)
            # loss1 = self.loss(logits1, targets, attention_mask=mask)
            # loss2 = self.loss(logits2, targets, attention_mask=mask)
            # loss3 = self.loss(logits3, targets, attention_mask=mask)
            # loss4 = self.loss(logits4, targets, attention_mask=mask)
            # loss5 = self.loss(logits5, targets, attention_mask=mask)
            # loss = (loss1 + loss2 + loss3 + loss4 + loss5) / 5
            output["loss"] = loss
        return output

In [ ]:
LOGGER = get_logger(CFG.OUTPUT_DIR+'train')

In [ ]:
seed_everything(42)

In [ ]:
train = pd.read_csv(CFG.DATA_PATH + 'train.csv')
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)
features = pd.read_csv(CFG.DATA_PATH + 'features.csv')
features = preprocess_features(features)
patient_notes = pd.read_csv(CFG.DATA_PATH + 'patient_notes.csv')

print(f"train.shape: {train.shape}")
print(f"features.shape: {features.shape}")
print(f"patient_notes.shape: {patient_notes.shape}")

train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')

train = clean_data(train)
train = clean_feature(train)

In [ ]:
train.head()

In [ ]:
ret = []
for x in train['location']:
    if x:
        ret.append([(int(s.split()[0]), int(s.split()[1]))
                    for s in x[0].split(';')])
    else:
        ret.append([])
train['target'] = ret

In [ ]:
tokenizer = get_tokenizer(CFG.model,
                          precompute=CFG.precompute_tokens,
                          df=train)

In [ ]:
Fold = GroupKFold(n_splits=5)
groups = train['pn_num'].values
for n, (train_index, val_index) in enumerate(
        Fold.split(train, train['location'], groups)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
train.groupby('fold').size()

In [ ]:
_oof_df = train_loop(train, 0, tokenizer, CFG, LOGGER)